This is going to be 3.3

# 3.3 Looking at the Lexical Vocabulary from the Perspective of the Literary Material

In section 3.2 we asked whether we can see differences between Old Babylonian literary compositions in their usage of vocabulary (lemmas and MWEs) attested in the lexical corpus. In this notebook we will change perspective and ask: are there particular lexical texts (or groups of lexical texts) that show a greater engagement with literary vocabulary than others?

In [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) we used Multiple Word Expressions, connecting words that are found in a lexical entry by underscrores (using `MWEtokenizer()` from the nltk module). The lemmas and MWE were used visualized in Venn diagrams to illustrate the intersection between lexical and literary vocabulary.

In this notebook we will use the ngram option of the `CountVectorizer()` function in order to find sequences of lemmas that are shared between lexical and literary texts. A ngram is a continuous sequence of *n* words (or lemmas). 


In large part, this notebook uses the same techniques and the same code as section 3.2 did, and the reader is referred there for further explanation. In some aspects, however, the process is different. In particular, we will use various aspects of `CountVectorizer()` and the related function `TfidfVectorizer()` to understand the relationship in more detail.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # this suppresses a warning about pandas from tqdm
import pandas as pd
from ipywidgets import interact
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.auto import tqdm
tqdm.pandas() # initiate pandas support in tqdm, allowing progress_apply() and progress_map()
import zipfile
import json
import numpy as np

Open the file `lexlines.p` which was produced in [3_1_Lit_Lex_Vocab.ipynb](./3_1_Lit_Lex_Vocab.ipynb). The file contains the pickled version of the DataFrame `lex_lines` in which the lexical ([dcclt](http://oracc.org/dcclt)) corpus is represented in line-by-line format.

In [ ]:
lex_lines = pd.read_pickle('output/lexlines.p')

### Special Case: OB Nippur Ura 6
The sixth chapter of the Old Babylonian Nippur version of the thematic list Ura deals with foodstuffs and drinks. This chapter was not standardized (each exemplar has its own order of items and sections) and therefore no composite text has been created in [DCCLT](http://oracc.org/dcclt). Instead, the "composite" of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) consists of the concatenation of all known Nippur exemplars of the list of foodstuffs. In our current dataframe, therefore, there are no lines where the field `id_text` equals "Q000043".

We create a "composite" by changing the field `id_text` in all exemplars of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) to "Q000043". 

In [ ]:
Ura6 = ["dcclt/P227657",
"dcclt/P227743",
"dcclt/P227791",
"dcclt/P227799",
"dcclt/P227925",
"dcclt/P227927",
"dcclt/P227958",
"dcclt/P227967",
"dcclt/P227979",
"dcclt/P228005",
"dcclt/P228008",
"dcclt/P228200",
"dcclt/P228359",
"dcclt/P228368",
"dcclt/P228488",
"dcclt/P228553",
"dcclt/P228562",
"dcclt/P228663",
"dcclt/P228726",
"dcclt/P228831",
"dcclt/P228928",
"dcclt/P229015",
"dcclt/P229093",
"dcclt/P229119",
"dcclt/P229304",
"dcclt/P229332",
"dcclt/P229350",
"dcclt/P229351",
"dcclt/P229352",
"dcclt/P229353",
"dcclt/P229354",
"dcclt/P229356",
"dcclt/P229357",
"dcclt/P229358",
"dcclt/P229359",
"dcclt/P229360",
"dcclt/P229361",
"dcclt/P229362",
"dcclt/P229365",
"dcclt/P229366",
"dcclt/P229367",
"dcclt/P229890",
"dcclt/P229925",
"dcclt/P230066",
"dcclt/P230208",
"dcclt/P230230",
"dcclt/P230530",
"dcclt/P230586",
"dcclt/P231095",
"dcclt/P231128",
"dcclt/P231424",
"dcclt/P231446",
"dcclt/P231453",
"dcclt/P231458",
"dcclt/P231742",
"dcclt/P266520"]
lex_lines.loc[lex_lines["id_text"].isin(Ura6), "id_text"] = "dcclt/Q000043"

# Computing text length
In order to evaluate the number of matches between a lexical text and the literary corpus we need a measure of text length. Text length is defined here as the number of lemmatized words in a text.

First the lines of `lit_lines` are aggregated to lexical compositions in the DataFrame `lex_comp`. 

In [ ]:
lex_comp = lex_lines.groupby(
    [lex_lines["id_text"]]).aggregate(
    {"lemma": ' '.join}).reset_index()

The function `lex_length()` computes the number of lemmas in each composition by first splitting the field `lemmas` into individual lemmas. A list comprehension removes all unlemmatized words, and the length of the resulting list is returned.

In [ ]:
def lex_length(lemmas):
    lemmas = lemmas.split()
    lemmas = [lemma for lemma in lemmas if not '[na]na' in lemma] # remove unlemmatized words
    length = len(lemmas)
    return length

First add the new field `length` by calling the function `lex_length()` for every row.

The DataFrame `lex_comp` has data from all Old Babylonian lexical texts currently in [dcclt](http://oracc.org/dcclt). Not all of these texts are lemmatized. In particular, documents that have been linked to a composite text are usually not lemmatized. Such documents have no lemmatized contents and therefore have length 0. These documents are removed from `lex_comp`.

In [ ]:
lex_comp['length'] = lex_comp['lemma'].progress_map(lex_length)
lex_comp = lex_comp.loc[lex_comp['length'] > 0] # remove compositions that have no lemmatized content

# Open list of Vocabulary Intersection
The file `lit_lex_vocab` is a list that includes all lemmas and Multiple Word Expressions that are shared by the literary corpus and the lexical corpus. This list was produced in [3_2_Lit_Lex.ipynb](./3_2_Lit_Lex.ipynb). In sections [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) lexical *entries* were turned into MWEs by connecting the individual lemmas by underscores (as in `amar\[young\]n_ga\[milk\]n_gu\[eat\]v/t`). In this notebook we will take a different approach by using ngrams. For that reason we need to replace all those underscores by spaces.

This vocabulary is used in the next section for building a Document Term Matrix.

In [ ]:
with open('output/lit_lex_vocab.txt', 'r', encoding = 'utf8') as l:
    lit_lex_vocab = l.read().splitlines()
lit_lex_vocab = [v.replace('_', ' ') for v in lit_lex_vocab]
lit_lex_vocab[:25]

### Document Term Matrix of Lexical Lines

The lexical corpus is transformed into a Document Term Matrix (or DTM), in the same way we did in [3.2](./3_2_Lit_Lex.ipynb) for the literary corpus - but with two important differences. 

First, the parameter `ngram_range` is set to (1, 5). With this parameter, `Countvectorizer()` will create a column for each word (ngram n=1), but also for each sequence of two words (bigram; n=2), or three words (trigram; n=3), etc. The entry `amar\[young\]n ga\[milk\]n gu\[eat\]v/t` ("calf that eats milk") will be represented as :

| type             | representation  |
|------------------|-----------------|
| unigram        | amar\[young\]n |
|                    | ga\[milk\]n |
|                    | gu\[eat\]v/t |
| bigram             | amar\[young\]n ga\[milk\]n |
|                    | ga\[milk\]n gu\[eat\]v/t |
| trigram | amar\[young\]n ga\[milk\]n gu\[eat\]v/t|

For longer entries we will also get 4-grams and 5-grams. 

A three word entry which was treated as a single unit in [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) now results in 6 potential columns in the Document Term Matrix. 

Potentially, this results in a very big (and very sparse) matrix. In order to limit the its size somewhat we use the vocabulary `lit_lex_vocab` which contains all lemmas and lexical entries shared by the lexical and literary corpora. These are the relevant vocabulary items that we wish to explore.

Second, we will take as our data the column `lemma` from the DataFrame `lex_lines` which represents the lexical corpus in *line* format, rather than in document format. This is important, because we do not want the to jump over line boundaries. The DTM will thus treat each line as a document. In the next step we will add up all the line-based data to create a proper DTM.

In [ ]:
cv = CountVectorizer(preprocessor = lambda x: x, tokenizer = lambda x: x.split(), vocabulary = lit_lex_vocab, ngram_range=(1, 5))
dtm = cv.fit_transform(lex_lines['lemma'])
lex_lines_dtm = pd.DataFrame(dtm.toarray(), columns= cv.get_feature_names(), index=lex_lines["id_text"])

### Aggregate DTM to Lexical Compositions
We may now reformat the line-based DTM into a true DTM by using the Pandas functions `groupby()` and `aggregate()`. The `aggregate()` function, in this case, is `sum`: for every word or ngram add the frequencies of all the lines of a single lexical composition.

Then the number of matches is computed. The field `n_matches` represents the number of unique words or ngrams that a lexical document shares with the literary corpus. For the code see [3.2](./3_2_Lit_Lex.ipynb) section 3.2.2.

In [ ]:
lex_comp_dtm = lex_lines_dtm.groupby('id_text').agg(sum).reset_index()
lex_comp_dtm["n_matches"] = lex_comp_dtm[lit_lex_vocab].astype(bool).sum(axis = 1)
lex_comp_dtm

# Remove Duplicates
Since the lexical data are drawn from multiple (sub)projects, it is possible that there are duplicates. Duplicates have the same P, Q, or X number. We select the version with the largest number of (lemmatized) words and drop others.

First the DataFrames `lex_comp_dtm` and `lex_comp` are merged on the field `id_text`. From the first DataFrame we only keep the fields `id_text` and `n_matches`, from the second one `id_text` and `length`. The merge method is `inner` so that documents that were omitted from `lex_comp` (because of length zero) do not show up again. Second, the field `id_text`, which has the format `dcclt/Q000041` or `dcclt/signlists/P447992`, is reduced to only the last 7 positions (P, Q, or X, followed by six digits). The merged DataFrame is ordered by length (from large to small) and, if duplicate `text_id`s are found, only the first one is kept with the Pandas method `drop_duplicates()`.

In [ ]:
lex = pd.merge(lex_comp_dtm[['id_text','n_matches']], lex_comp[['id_text', 'length']], on = 'id_text', how = 'inner')
lex['id_text'] = lex['id_text'].str[-7:]
lex = lex.sort_values(by = 'length', ascending=False)
lex = lex.drop_duplicates(subset = 'id_text', keep = 'first')

### Adding Metadata
The metadata of the lexical texts (such as composition name, etc.) is found in the JSON files for each of the (sub)projects downloaded in section [3.1](./3_1_Lit_Lex_Vocab.ipynb). The code is essentially the same as in [3.2](./3_2_Lit_Lex.ipynb), but since there are multiple projects involved, it is done in a loop.

In [ ]:
cat = {}
for proj in ['dcclt', 'dcclt/signlists', 'dcclt/nineveh', 'dcclt/ebla']:
    f = proj.replace('/', '-')
    file = f"jsonzip/{f}.zip" # The ZIP file was downloaded in notebook 3_1
    z = zipfile.ZipFile(file) 
    st = z.read(f"{proj}/catalogue.json").decode("utf-8")
    j = (json.loads(st))
    cat.update(j["members"])
cat_df = pd.DataFrame(cat).T
cat_df["id_text"] = cat_df["id_text"].fillna(cat_df["id_composite"])
cat_df = cat_df.fillna('')
cat_df = cat_df[["id_text", "designation", "subgenre"]]

### Merging
Now merge `cat_df` with the DataFrame `lex` on the field `id_text`.

In [ ]:
lex = pd.merge(cat_df, lex, on = 'id_text', how = 'inner')

### Normalizing
Long lexical documents have more matches than short one. Normalize by dividing the number of matches (`n_matches`) by text length. For very short documents this measure has little value, only display longer documents.

In [ ]:
lex['norm'] = lex['n_matches'] / lex['length']
lex = lex.sort_values(by = 'norm', ascending = False)
lex.loc[lex.length > 250]

### Explore the Results
Explore the results in an inteactive table.

In [ ]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex2 = lex.copy()
lex2['id_text'] = [anchor.format(val,val) for val in lex['id_text']]

In [ ]:
@interact(sort_by = lex2.columns, rows = (1, len(lex2), 1), min_length = (1,500,5))
def sort_df(sort_by = "norm", ascending = False, rows = 25, min_length = 250):
    return lex2.loc[lex2.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style

### Discussion
When order by `norm` the top of the list is formed by lexical texts such as OB Nippur Ea, OB Nippur Lu, OB Nippur Izi, OB Nippur Kagal, and OB Nippur Kagal. If we restricted the DataFrame to composites (Q numbers) only, this would come out even clearer. All the lexical texts belong to what Jay Crisostomo has labeled "ALE": Advanced Lexical Exercises. These exercises belong to the advanced first stage of education, just before students would start copying literary texts. The thematic lists collected in Ura (lists of trees, wooden objects, reeds, reed objects, clay, pottery, hides, metals and metal objects, animals, meat cuts, fish, birds, plants, etc.) have much lower `norm` values and thus less overlap with literary vocabulary.

### Import Words: TF-IDF
What the above analysis does not address is the importance of the matching words. Words such as **igi\[eye\]n** or **šag\[heart\]n** are used in many kinds of expressions and will be found in all literary compositions - except for the most fragmentary ones. Those words are matched by the list Ugumu (list of body parts) but such matches are much less meaningful than a rarer word or expression like **nir\[stone\]n babbardil\[~stone\]n** (a type of agate with a single white stripe) which is fairly common in the lexical tradition, but only appears in one place in the literary corpus ([UET 6/3 561](http://oracc.org/epsd2/literary/P346598)) and with some frequency in administrative documents.

In order to distinguish between more and less important matches we will compute the [tf-idf](https://en.wikipedia.org/wiki/Tf-idf) value for each word or expression in each literary document. Tf-idf is short for term frequency - inverse document frequency and is a widely used measure (or rather a class of measures) for weighing the importance of a word in a document. Term frequency refers to the frequency of a term in a document; a term that is more frequently used is probably important in that document. Document frequency refers to the number of documents in the corpus in which the term is found. A term that is found in the great majority of documents has little distinguishing value. A high document frequency, therefore, results in a lower relevance of the term. Thus the tf-idf value of words like **igi\[eye\]n** or **šag\[heart\]n** in any document is expected to be low - even though they may appear very frequently, because they appear in almost every document. On the other hand, the tf-idf value of **nir\[stone\]n babbardil\[~stone\]n** in [UET 6/3 561](http://oracc.org/epsd2/literary/P346598) is going to be fairly low, too, because the word appears only once in that text. A high value is associated with high frequency in a few documents. 

We can compute tf-idf values with `TfidfVectorizer()` from the `sklearn` package. `TfidfVectorizer()` is a close relative of `CountVectorizer()` and works in essentially the same way.

In [ ]:
lit_lines = pd.read_pickle('output/litlines.p')
lit_comp = lit_lines.groupby(['id_text']).agg({'lemma' : ' '.join}).reset_index()
lit_comp['id_text'] = lit_comp['id_text'].str[-7:]
tv = TfidfVectorizer(tokenizer = lambda x: x.split(), preprocessor = lambda x: x, ngram_range = (1,5), vocabulary =lit_lex_vocab)
dtm = tv.fit_transform(lit_comp['lemma'])
lit_df = pd.DataFrame(dtm.toarray(), columns= tv.get_feature_names(), index=lit_comp["id_text"])
lit_df

We can compute the mean tf-idf value of each word by dividing the sum of each column (the total tf-idf weights for a word or expression) by the number of times that the word is attested at least once in a document (the number of non-zero cells in a column). This results in an array of values for each column in the DTM.

> Note: or rather divide by the number of rows?

In [ ]:
mean = lit_df[lit_lex_vocab].values.sum(axis=0) / len(lit_df) #lit_df[lit_lex_vocab].astype(bool).values.sum(axis=0) #total weights by total hits
mean

The array `mean` indicates the mean weight of each word or expression in the literary corpus. We can multiply the entire lexical DTM with that array.

In [ ]:
lit_lex_tfidf = lex_comp_dtm.copy()
lit_lex_tfidf[lit_lex_vocab] = lit_lex_tfidf[lit_lex_vocab].mul(mean, axis = 1)

In [ ]:
lit_lex_tfidf['id_text'] = lit_lex_tfidf['id_text'].str[-7:]

In [ ]:
lit_lex_tfidf['weighted'] = lit_lex_tfidf[lit_lex_vocab].sum(axis=1)

In [ ]:
lit_lex_tfidf

In [ ]:
lit_lex_tfidf.shape

> Note we probably still need to remove duplicates here? Try to merge with lex and have columns `norm`and `norm_tfidf`.

In [ ]:
lex2 = pd.merge(cat_df, lit_lex_tfidf[['weighted', 'id_text']], on = 'id_text', how = 'inner')
lex2 = pd.merge(lex2, lex[['length', 'n_matches', 'id_text']], on = 'id_text', how = 'inner')

Instead of dividing by length look at mean value of weighted
```python
lex2['norm'] = lex2['weigthed'] / lex2.astype(bool).sum(axis = 1)
```

In [ ]:
#lex2['norm'] = lex2['weighted'] / lex2['n_matches']
lex2['norm'] = lex2['weighted'] / lex2['length']
lex2.sort_values(by = 'norm', ascending = False)

In [ ]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex3 = lex2.copy()
lex3['id_text'] = [anchor.format(val,val) for val in lex2['id_text']]

In [ ]:
@interact(sort_by = lex3.columns, rows = (1, len(lex3), 1), min_length = (1,500,5))
def sort_df(sort_by = "norm", ascending = False, rows = 25, min_length = 200):
    return lex3.loc[lex3.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style